<a href="https://colab.research.google.com/github/gatherheart/Recommendation_With_Buffalo/blob/main/KAKAO_Buffalo_CF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install cython
!pip install n2
!pip install buffalo

     |████████████████████████████████| 8.6MB 5.8MB/s 
  Created wheel for n2: filename=n2-0.1.7-cp36-cp36m-linux_x86_64.whl size=2311625 sha256=bbb519071692bf1dd970159e1ada26a44dd3ad635e11cfa515e5c9839cea8065
  Stored in directory: /root/.cache/pip/wheels/a9/ae/b5/1116339d5b0cd43dd677840e5f9553c9fd73da9a41fb4e2bd7
Successfully built n2
     |████████████████████████████████| 2.6MB 9.0MB/s 
  Created wheel for buffalo: filename=buffalo-1.2.0-cp36-cp36m-linux_x86_64.whl size=8526301 sha256=c74f6972d27c890c8dcdfca866218ec92bcc446b9c57636884c8b95f0d2a3548
  Stored in directory: /root/.cache/pip/wheels/c1/b3/2e/afd93810d8689bd04e260a1d7ec98375da8e7c8ec5f6269e4b
Successfully built buffalo


In [ ]:
!mkdir data
!wget -q http://www.grouplens.org/system/files/ml-1m.zip ./data
!unzip -o ml-1m -d data

Archive:  ml-1m.zip
   creating: data/ml-1m/
  inflating: data/ml-1m/movies.dat   
  inflating: data/ml-1m/ratings.dat  
  inflating: data/ml-1m/README       
  inflating: data/ml-1m/users.dat    


In [ ]:
import numpy as np
import pandas as pd
from scipy.io import mmwrite
from scipy.io import mmread
from scipy.sparse import csr_matrix

In [22]:
ratings = pd.read_csv("data/ml-1m/ratings.dat", header=None, sep="::", engine='python')
ratings.columns = ["uid", "iid", "rating", "timestamp"]

In [23]:
movies = pd.read_csv('data/ml-1m/movies.dat', header=None, sep="::", engine='python')
movies.columns = ['iid', 'movie_name', 'genre']

In [24]:
def parse_moviename(movie_name):
    return movie_name.replace(' ', '_').encode('utf-8').decode('ascii', 'ignore')

In [25]:
iid_to_movie_name = dict(zip(movies.iid.tolist(), movies.movie_name.tolist()))
iid_to_movie_name = {iid: parse_moviename(movie_name) for (iid, movie_name) in iid_to_movie_name.items()}

In [26]:
uid_to_idx = {uid: idx for (idx, uid) in enumerate(ratings.uid.unique().tolist())}
iid_to_idx = {iid: idx for (idx, iid) in enumerate(ratings.iid.unique().tolist())}
idx_to_movie_name = {idx:iid_to_movie_name[iid] for (iid, idx) in iid_to_idx.items()}

In [27]:
print("Examples of movie names\n")

for i in range(30, 35):
    print("[index %d] movie_name: %s" % (i, idx_to_movie_name[i]))

Examples of movie names

[index 30] movie_name: Antz_(1998)
[index 31] movie_name: Girl,_Interrupted_(1999)
[index 32] movie_name: Hercules_(1997)
[index 33] movie_name: Aladdin_(1992)
[index 34] movie_name: Mulan_(1998)


In [28]:
row, col, dat = ratings.uid.tolist(), ratings.iid.tolist(), ratings.rating.tolist()
row = [uid_to_idx[r] for r in row]
col = [iid_to_idx[c] for c in col]

In [29]:
train_matrix = csr_matrix((dat, (row,col)), shape=(1 + np.max(row), 1 + np.max(col)))
print(train_matrix.shape)

(6040, 3706)


In [30]:
mmwrite('data/ml-1m/main', train_matrix)

In [31]:
with open("data/ml-1m/uid", "w") as f:
    for uid in uid_to_idx:
        print(uid, file=f)

with open("data/ml-1m/iid", "w") as f:
    for iid, movie_name in idx_to_movie_name.items():
        print(movie_name, file=f)

In [32]:
ratings_as_list = ratings.sort_values(by='timestamp').groupby('uid').iid.apply(list).reset_index()
uid = ratings_as_list.uid.tolist()
seen_iids = ratings_as_list.iid.tolist()

In [33]:
seen_iids = [' '.join([iid_to_movie_name[iid] for iid in iids]) for iids in seen_iids]

In [34]:
print(seen_iids[0])

Girl,_Interrupted_(1999) Titanic_(1997) Back_to_the_Future_(1985) Cinderella_(1950) Meet_Joe_Black_(1998) Last_Days_of_Disco,_The_(1998) Erin_Brockovich_(2000) To_Kill_a_Mockingbird_(1962) Christmas_Story,_A_(1983) Star_Wars:_Episode_IV_-_A_New_Hope_(1977) Wallace_&_Gromit:_The_Best_of_Aardman_Animation_(1996) One_Flew_Over_the_Cuckoo's_Nest_(1975) Wizard_of_Oz,_The_(1939) Fargo_(1996) Run_Lola_Run_(Lola_rennt)_(1998) Rain_Man_(1988) Saving_Private_Ryan_(1998) Awakenings_(1990) Gigi_(1958) Sound_of_Music,_The_(1965) Driving_Miss_Daisy_(1989) Mary_Poppins_(1964) Bambi_(1942) Apollo_13_(1995) E.T._the_Extra-Terrestrial_(1982) My_Fair_Lady_(1964) Ben-Hur_(1959) Big_(1988) Dead_Poets_Society_(1989) Sixth_Sense,_The_(1999) James_and_the_Giant_Peach_(1996) Ferris_Bueller's_Day_Off_(1986) Secret_Garden,_The_(1993) Toy_Story_2_(1999) Airplane!_(1980) Dumbo_(1941) Pleasantville_(1998) Princess_Bride,_The_(1987) Snow_White_and_the_Seven_Dwarfs_(1937) Miracle_on_34th_Street_(1947) Ponette_(1996) 

In [35]:
with open("data/ml-1m/stream", "w") as f:
    for iid_list in seen_iids:
        print(iid_list, file=f)

In [ ]:
from buffalo.algo.als import ALS
from buffalo.algo.bpr import BPRMF
from buffalo.misc import aux, log
from buffalo.algo.options import ALSOption, BPRMFOption
import buffalo.data
from buffalo.data.mm import MatrixMarketOptions

log.set_log_level(1) # set log level 3 or higher to check more information

In [ ]:
MODEL_TO_USE = "ALS"
MODEL_TO_USE = "BPR"# un-comment this if you want to use BPR

In [ ]:
if MODEL_TO_USE == "ALS":
    opt = ALSOption().get_default_option()  
elif MODEL_TO_USE == "BPR":
    opt = BPRMFOption().get_default_option()

In [ ]:
opt.evaluation_on_learning =  True
opt.validation = aux.Option({'topk': 10})

In [ ]:
opt

{'accelerator': False,
 'beta1': 0.9,
 'beta2': 0.999,
 'compute_loss_on_training': True,
 'd': 20,
 'data_opt': {},
 'early_stopping_rounds': 0,
 'eps': 1e-10,
 'evaluation_on_learning': True,
 'evaluation_period': 100,
 'hyper_threads': 256,
 'lr': 0.002,
 'min_lr': 0.0001,
 'model_path': '',
 'num_iters': 100,
 'num_negative_samples': 1,
 'num_workers': 1,
 'optimizer': 'sgd',
 'per_coordinate_normalize': False,
 'random_positive': False,
 'random_seed': 0,
 'reg_b': 0.025,
 'reg_i': 0.025,
 'reg_j': 0.025,
 'reg_u': 0.025,
 'sampling_power': 0.0,
 'save_best': False,
 'save_period': 10,
 'update_i': True,
 'update_j': True,
 'use_bias': True,
 'validation': {'topk': 10},
 'verify_neg': True}

In [ ]:
data_opt = MatrixMarketOptions().get_default_option()

In [ ]:
data_opt.input.main = 'data/ml-1m/main.mtx'

In [ ]:
data_opt.input.iid = 'data/ml-1m/iid'
data_opt.input.uid = 'data/ml-1m/uid'

In [ ]:
data_opt

{'data': {'batch_mb': 1024,
  'internal_data_type': 'matrix',
  'path': './mm.h5py',
  'tmp_dir': '/tmp/',
  'use_cache': False,
  'validation': {'max_samples': 500, 'name': 'sample', 'p': 0.01}},
 'input': {'iid': 'data/ml-1m/iid',
  'main': 'data/ml-1m/main.mtx',
  'uid': 'data/ml-1m/uid'},
 'type': 'matrix_market'}

In [36]:
data = buffalo.data.load(data_opt)
data.create()

In [37]:
if MODEL_TO_USE == "ALS":
    model = ALS(opt, data=data)
elif MODEL_TO_USE == "BPR":
    model = BPRMF(opt, data=data)
model.initialize()